In [37]:
import os
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from scipy.signal import argrelextrema
from scipy.special import ndtr
from arch import arch_model

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split, TimeSeriesSplit
from sklearn.base import BaseEstimator, TransformerMixin, clone
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    make_scorer, accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, 
    roc_curve, RocCurveDisplay, auc, roc_auc_score, precision_recall_curve, PrecisionRecallDisplay, precision_score, recall_score, 
)
from sklearn.utils.class_weight import compute_class_weight
from xgboost import XGBClassifier, plot_importance

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

import phik
from phik.report import plot_correlation_matrix
import fAux
import mc_permutation
import detrendPrice 


import ta
from ta.trend import EMAIndicator 
global global_returns
global global_labels

class FeatureDropper(BaseEstimator, TransformerMixin):
    def __init__(self, drop_cols):
        self.drop_cols = drop_cols

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.drop(columns=self.drop_cols)
    


# ---
def get_data(ticker, start_date='1999-01-01', end_date=None, save_csv=False):
    os.makedirs("data", exist_ok=True)
    filename = f"data/{ticker.lower()}.csv"
    # if os.path.exists(filename):
    #     print(f"Data for {i} already exists, skipping download.")
    #     continue

    data = yf.download(ticker.upper(), start=start_date, end=end_date, auto_adjust=True)

    # Handle empty or failed download
    if data.empty:
        print(f"No data found for {ticker}")
        return pd.DataFrame()
    
    # Flatten column headers if it's a MultiIndex (e.g., from group_by='ticker')
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.get_level_values(0)
    
    data['Ticker'] = ticker.lower()
    data.index = pd.to_datetime(data.index)
    data.columns = [col.lower() for col in data.columns]


    if save_csv:
        data.to_csv(filename, index=False)
        print(f"Saved data for {ticker} to {filename}")
        
    return data



# model for longing; trading at open tomorrow (t+1)
# based on today's data (ohlcv and more), decide whether to long tomorrow
# binary target: 1 if price reverts up (above the threshold e.g. gain over 2%) wihtin the next five trading days, 0 otherwise
def create_target_long(df, threshold=0.02, lookahead=5):
    """
    Label as 1 if max return over the next 'lookahead' days is >= threshold.
    Else, label as 0.
    """
    # df = df.reset_index().rename(columns={'Date': 'date'})
    df['price_tmrw'] = df['open'].shift(-1) # trade at open next day

    df['seq_index'] = range(len(df))
    future_max_list = []
    for i in df['seq_index']:
        if i + 1 + lookahead <= len(df):
            window = df['open'].iloc[i+1 : i+1+lookahead]
            future_max_list.append(window.max())
        else:
            future_max_list.append(np.nan)

    df['future_max'] = future_max_list

    df['future_return'] = (df['future_max'] - df['price_tmrw']) / df['price_tmrw']
    df['target_long'] = (df['future_return'] >= threshold).astype(int)
    df.drop(columns=['seq_index', 'price_tmrw', 'future_max', 'future_return'], inplace=True)
    return df



def feature_engineering(df): # for trading at open tomorrow
    # original_feats = df.columns

    # phase 1: add features thats fundamental to mean reversion trading with bollinger bands and rsi
    df['sma30'] = df['close'].rolling(30).mean()
    df['sma10'] = df['close'].rolling(10).mean()
    df['sma_diff'] = df['sma10'] - df['sma30']
    df['sma_slope'] = df['sma10'].diff()
    df['std30'] = df['close'].rolling(30).std()
    df['bollinger_upper'] = df['sma30'] + 2 * df['std30']
    df['bollinger_lower'] = df['sma30'] - 2 * df['std30']
    df['percent_b'] = (df['close'] - df['bollinger_lower']) / (df['bollinger_upper'] - df['bollinger_lower'])
    df['bollinger_z'] = (df['close'] - df['sma30']) / df['std30']
    df['price_near_lower_bb'] = (df['close'] <= df['bollinger_lower'] * 1.01).astype(int)

    df['rsi14'] = ta.momentum.RSIIndicator(df['close'], window=14).rsi()    
    df['prod_bollingerz_rsi'] = df['percent_b'] * df['rsi14']

    df['rsi_smooth'] = df['rsi14'].rolling(3).mean()
    # Detect local lows
    rsi_vals = df['rsi_smooth'].values
    local_lows = argrelextrema(rsi_vals, np.less, order=5)[0]

    df['rsi_local_low'] = 0
    df.iloc[local_lows, df.columns.get_loc('rsi_local_low')] = 1

    # phase 2: add other technical features  
    df['daily_return'] = df['open'].pct_change()
    df['rolling_volatility14'] = df['daily_return'].rolling(window=30).std()
    df['atr'] = ta.volatility.AverageTrueRange(high=df['high'], low=df['low'], close=df['close']).average_true_range()

    # spy_ret5 (discard for now; if wanting to add add it from feature_engineering_add_macro)
    # spy_ret5 = df_spy['close'].pct_change(periods=5)
    # df = df.merge(spy_ret5.rename('spy_ret5'), left_index=True, right_index=True, how='left')

    # phase 3: more features!
    
    # 3.1 GARCH(1,1) on returns
    returns = df['close'].pct_change().dropna() * 100 # in percent
    am = arch_model(returns, vol='Garch', p=1, q=1)
    res = am.fit(disp='off')
    df['garch_vol'] = res.conditional_volatility # in percent

    # 3.2 tiem related features
    df['year'] = df.index.year
    df['month'] = df.index.month
    df['week'] = df.index.isocalendar().week
    df['dayofweek'] = df.index.dayofweek

    # df.dropna(inplace=True)
    # new_feats = [i for i in df.columns if i not in original_feats]
    return df



def feature_engineering_add_macro(df,etf:str):
    df_etf = get_data(etf, end_date = df.index.max())
    df_etf['sma10'] = df_etf['close'].rolling(10).mean()
    df_etf['sma20'] = df_etf['close'].rolling(20).mean()
    df_etf['sma50'] = df_etf['close'].rolling(50).mean()
    df_etf['sma200'] = df_etf['close'].rolling(200).mean()
    df_etf['trend_10_50'] = (df_etf['sma10'] > df_etf['sma50']).astype(int)
    df_etf['trend_20_50'] = (df_etf['sma20'] > df_etf['sma50']).astype(int)
    df_etf['trend_50_200'] = (df_etf['sma50'] > df_etf['sma200']).astype(int)
    df = df.merge(df_etf['trend_10_50'].rename(f'{etf}_trend_10_50'), left_index=True, right_index=True, how='left')
    df = df.merge(df_etf['trend_20_50'].rename(f'{etf}_trend_20_50'), left_index=True, right_index=True, how='left')
    df = df.merge(df_etf['trend_50_200'].rename(f'{etf}_trend_50_200'), left_index=True, right_index=True, how='left')
    return df



def plot_model_metrics(y_true, y_proba):
    y_pred = (y_proba >= 0.5).astype(int)
    fig, axes = plt.subplots(1, 3, figsize=(14, 4))

    # 1. Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(ax=axes[0], colorbar=False)

    # 2. ROC Curve
    fpr, tpr, _ = roc_curve(y_true, y_proba)
    roc_auc = auc(fpr, tpr)
    roc_disp = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc)
    roc_disp.plot(ax=axes[1])

    # 3. Precision-Recall Curve
    precision, recall, _ = precision_recall_curve(y_true, y_proba)
    pr_disp = PrecisionRecallDisplay(precision=precision, recall=recall)
    pr_disp.plot(ax=axes[2])

    plt.tight_layout()
    plt.show()



def eval_pipe(X, y, pipe, n_split=5, predict_proba_threshold=0.5):
    '''
    for evaluations on train set, make sure X and y does not contain anything from the test set (future data), 
    and use n_split=5 for 5-fold time series cross-validation
    '''
    tscv = TimeSeriesSplit(n_splits=n_split)
    for i, (train_index, test_index) in enumerate(tscv.split(X)):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        pipe.fit(X_train, y_train)
        y_proba = pipe.predict_proba(X_test)[:, 1]
        y_pred = (y_proba >= predict_proba_threshold).astype(int)

        print(f'Fold {i+1}:\n{classification_report(y_test, y_pred)}')

        if i == 0:
            print('Fold 1 visualized below')
            plot_model_metrics(y_test, y_proba)

def pred_proba_to_signal(y_proba, threshold=0.5):
    """
    Convert predicted probabilities to binary signals.
    1 if probability >= threshold, else 0.
    returns a Series of signals.
    """
    return (y_proba >= threshold).astype(int)


def entry_exit(df, use_vol=None, take_profit=1, stop_loss=1): 
    '''
    FOR EACH INSTRUMENT (don't use the aggregated one that contains multiple tickers)
    takes in the X_test or ohlcv dataframe containing model signals,
    returns a df that contains entry and exit dates, prices, returns, and holding days
    use_vol: either 'atr' or 'garch_vol'
    take_profit and stop loss as percentage
    '''
    # if date as index
    if pd.api.types.is_datetime64_any_dtype(df.index): 
        df = df.reset_index().rename(columns={'Date': 'date'})

    trades = []
    i = 0
    n = len(df)
    if use_vol not in [None, 'atr', 'garch_vol']:
        raise ValueError("use_vol must be one of: None, 'atr' or 'garch_vol'")
    
    if use_vol == 'atr':
        multiplier = df['atr']
    elif use_vol == 'garch_vol':
        multiplier = df['garch_vol']
    else:
        multiplier = pd.Series(1, index=df.index)

    while i < n - 6:  # we need at least 5 days ahead, plus trading at open tomorrow
        if df['model_signal'].iloc[i] == 1:
            entry_date = df['date'].iloc[i+1]
            entry_price = df['open'].iloc[i+1]
            exit_price = None
            exit_date = None
            holding = None
            exit_reason = None

            for j in range(1, 6):  # check up to 5 days ahead
                if i + 1 + j >= n:
                    break

                next_price = df['open'].iloc[i + 1 + j]
                ret = (next_price - entry_price) / entry_price

                # Exit Conditions
                if ret >= take_profit * 0.01 * multiplier.iloc[i]:  # profit target
                    exit_price = next_price
                    exit_date = df['date'].iloc[i + 1 + j]
                    holding = j
                    exit_reason = 'profit_target'
                    break
                elif ret <= -stop_loss * 0.01 * multiplier.iloc[i]:  # stop loss
                    exit_price = next_price
                    exit_date = df['date'].iloc[i + 1 + j]
                    holding = j
                    exit_reason = 'stop_loss'
                    break
                elif df['open'].iloc[i + 1 + j] >= df['sma30'].iloc[i + 1 + j]:  # revert to SMA30
                    exit_price = next_price
                    exit_date = df['date'].iloc[i + 1 + j]
                    holding = j
                    exit_reason = 'revert_to_sma30'
                    break

            if exit_price is None:
                # Max holding (5th day)
                exit_price = df['open'].iloc[i + 1 + 5]
                exit_date = df['date'].iloc[i + 1 + 5]
                holding = 5

            trade_return = (exit_price - entry_price) / entry_price
            trades.append({
                'entry_date': entry_date,
                'exit_date': exit_date,
                'entry_price': entry_price,
                'exit_price': exit_price,
                'return': trade_return,
                'holding_days': holding,
                'exit_reason': exit_reason
            })

            i = i + holding  # skip to the day after exit
        else:
            i += 1

    return pd.DataFrame(trades)



def backtest(X_test, model, proba_threshold = 0.5, use_vol=None, take_profit=1, stop_loss=1):
    trade_stats = {} 

    y_proba = model.predict_proba(X_test)[:, 1]
    # print(f'max proba: {y_proba.max()}')
    signals = pred_proba_to_signal(y_proba, threshold=proba_threshold)
    X_test_signal = X_test.assign(model_signal=signals, y_proba=y_proba)

    trades = entry_exit(X_test_signal, use_vol, take_profit, stop_loss)

    total_holding_days = trades['holding_days'].sum()
    total_trading_days = len(X_test_signal)
    holding_time_percentage = total_holding_days/total_trading_days
    n_trades = len(trades)
    n_wins = len(trades[trades['return'] > 0])
    win_rate = n_wins/n_trades

    # print(trades['exit_reason'].value_counts()) # 3 possible outcomes: profit_target, stop_loss, revert_to_sma30
    # print('holding days spread:')
    # print(trades['holding_days'].value_counts())
    # print('---')
    # print(f"total holding days: {total_holding_days} over {total_trading_days} trading days ({holding_time_percentage:.2%})")
    # print(f'number of trades: {n_trades}')
    # print(f"win rate: {win_rate:.2%}")

    trade_stats['max_proba'] = y_proba.max()
    trade_stats['exit_reason_spread'] = trades['exit_reason'].value_counts()
    trade_stats['holding_days_spread'] = trades['holding_days'].value_counts()
    trade_stats['total_holding_days'] = total_holding_days
    trade_stats['total_trading_days'] = total_trading_days
    trade_stats['holding_time_percentage'] = holding_time_percentage
    trade_stats['n_trades'] = n_trades
    trade_stats['n_wins'] = n_wins
    trade_stats['win_rate'] = win_rate

    return trades, trade_stats



def get_cagr(df):
    start_date = df['entry_date'].min()
    end_date = df['exit_date'].max()
    n_years = (end_date - start_date).days / 365.25
    capital = 1
    for r in df['return']:
        capital *= (1 + r)
    cagr = capital ** (1 / n_years) - 1
    return cagr

def get_sharpe(df_trades, total_trading_days):
    ret_mean = df_trades['return'].mean()
    ret_std = df_trades['return'].std()
    n_trades = len(df_trades)
    trades_per_year = (n_trades/total_trading_days) * 252
    return (ret_mean/ret_std) * np.sqrt(trades_per_year)


def check_return_stats(df):
    print(f"check return stats: \n {df['return'].describe()}")
    sns.histplot(df['return'])



def print_trade_info(ticker, dfs=dfs):
    print(
        f"ticker: {ticker} \n"
        f"total_holding_days: {dfs[ticker]['trades']['trade_stats']['total_holding_days']} \n"
        f"total_trading_days: {dfs[ticker]['trades']['trade_stats']['total_trading_days']} \n"
        f"holding_time_percentage: {dfs[ticker]['trades']['trade_stats']['holding_time_percentage']:.2%} \n" 
        f"n_trades: {dfs[ticker]['trades']['trade_stats']['n_trades']} \n" 
        f"win_rate: {dfs[ticker]['trades']['trade_stats']['win_rate']:.2%} \n" 
        f"cagr: {dfs[ticker]['trades']['cagr']:.2%} \n" 
        f"sharpe: {dfs[ticker]['trades']['sharpe']:.3} \n" 
        "---"
    )


In [8]:
# first 25 stocks are the top 25 holdings in qqq as of jul 17 2025
# plus a couple others from my watchlist
stocks = [
    'nvda','msft','aapl','amzn','avgo',
    'meta','nflx','tsla','googl','cost',
    'goog','pltr','csco','amd','tmus',
    'lin','intu','pep','txn','isrg',
    'bkng','qcom','amgn','adbe','amat',
    'spot','tsm','rddt','mstr','rklb',
]
dfs_raw = {i: get_data(i,end_date='2025-07-17') for i in stocks}

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [9]:
dfs = {}
for key, value in dfs_raw.items():
    dfs[key] = {}
    this_df = value.copy()
    dfs[key]['ohlcv'] = this_df
    this_df = create_target_long(this_df)
    this_df = feature_engineering(this_df)
    this_df = feature_engineering_add_macro(this_df, 'spy')
    this_df = feature_engineering_add_macro(this_df, 'qqq')
    this_df.dropna(inplace=True)
    X = this_df.drop(columns=['target_long','ticker'])
    y = this_df['target_long']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)
    dfs[key]['X_train'] = X_train
    dfs[key]['X_test'] =X_test
    dfs[key]['y_train'] = y_train
    dfs[key]['y_test'] = y_test

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [12]:
drop_feats = [
    # 'date',
    'high','low','sma30','sma10','std30',
    'bollinger_upper','bollinger_lower',
    'rsi_smooth',
]

ct = make_column_transformer(
    ('drop', drop_feats),  
    remainder='passthrough'  
)

pipe = Pipeline([
    ('ct', ct),
    ('xgb', XGBClassifier(
        objective='binary:logistic',
        eval_metric='logloss',
        use_label_encoder=False,
        random_state=42
    ))
])

# param_grid = {
#     'xgb__max_depth': [3, 5, 7],
#     'xgb__learning_rate': [0.01, 0.05, 0.1],
#     'xgb__n_estimators': [100, 300, 500],
#     'xgb__subsample': [0.5, 0.7, 1.0],
#     'xgb__colsample_bytree': [0.5, 0.7, 1.0]
# }

# smaller param grid
param_grid = {
    'xgb__max_depth': [3, 5],                  # shallower trees generalize better
    'xgb__learning_rate': [0.05, 0.1],         # fast vs. slow learner
    'xgb__n_estimators': [100, 300],           # fewer rounds is faster
    'xgb__subsample': [0.7, 1.0],              # some regularization
    'xgb__colsample_bytree': [0.7, 1.0]        # helps reduce overfitting
}


rscv = RandomizedSearchCV(
    pipe,
    param_distributions=param_grid,
    n_iter=30,  # number of random combinations to try
    scoring='f1',  # or 'roc_auc', 'accuracy', etc.
    cv=5,
    verbose=1,
    n_jobs=-1,
    random_state=42
)

In [19]:
for key in dfs.keys():
    this_rscv = clone(rscv)
    this_rscv.fit(dfs[key]['X_train'], dfs[key]['y_train'])
    dfs[key]['best_estimator'] = this_rscv.best_estimator_
    dfs[key]['best_params'] = this_rscv.best_params_
    dfs[key]['best_score'] = this_rscv.best_score_
    # backtest with proba_threshold = 0.7, garch_vol as multiplier, 1.5 to 1 risk reward ratio
    this_trades, trade_stats = backtest(
        dfs[key]['X_test'], 
        this_rscv, 
        proba_threshold=0.7, 
        use_vol = 'garch_vol',
        take_profit=1.5,
        stop_loss=1
    )
    dfs[key]['trades'] = {}
    dfs[key]['trades']['df_trade'] = this_trades
    dfs[key]['trades']['trade_stats'] = trade_stats
    # cagr and sharpe
    dfs[key]['trades']['cagr'] = get_cagr(this_trades)
    dfs[key]['trades']['sharpe'] = get_sharpe(
        this_trades, 
        dfs[key]['trades']['trade_stats']['total_trading_days']
    )
    # break

Fitting 5 folds for each of 30 candidates, totalling 150 fits


/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [17:58:04] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [17:58:04] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [17:58:04] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [17:58:04] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not

Fitting 5 folds for each of 30 candidates, totalling 150 fits


/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [17:58:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [17:58:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [17:58:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [17:58:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not

Fitting 5 folds for each of 30 candidates, totalling 150 fits


...

/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [18:00:08] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [18:00:08] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [18:00:08] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [18:00:08] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not

Fitting 5 folds for each of 30 candidates, totalling 150 fits


/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [18:00:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [18:00:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [18:00:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [18:00:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not

Fitting 5 folds for each of 30 candidates, totalling 150 fits


/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [18:00:17] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [18:00:17] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [18:00:17] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [18:00:17] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not

In [35]:
for idx, i in enumerate(list(dfs.keys())):
    print(f"{i}", end='  ')
    if idx % 5 == 4:
        print()  # new line after every 5 keys

nvda  msft  aapl  amzn  avgo  
meta  nflx  tsla  googl  cost  
goog  pltr  csco  amd  tmus  
lin  intu  pep  txn  isrg  
bkng  qcom  amgn  adbe  amat  
spot  tsm  rddt  mstr  rklb  


In [38]:
print_trade_info('spot')
print_trade_info('pltr')
print_trade_info('tsla')
print_trade_info('tmus')
print_trade_info('rklb')
print_trade_info('mstr')
print_trade_info('rddt')


ticker: spot 
total_holding_days: 192 
total_trading_days: 361 
holding_time_percentage: 53.19% 
n_trades: 162 
win_rate: 56.17% 
cagr: 75.07% 
sharpe: 1.63 
---
ticker: pltr 
total_holding_days: 10 
total_trading_days: 235 
holding_time_percentage: 4.26% 
n_trades: 8 
win_rate: 62.50% 
cagr: 15.47% 
sharpe: 0.944 
---
ticker: tsla 
total_holding_days: 85 
total_trading_days: 751 
holding_time_percentage: 11.32% 
n_trades: 43 
win_rate: 79.07% 
cagr: 52.10% 
sharpe: 2.01 
---
ticker: tmus 
total_holding_days: 43 
total_trading_days: 912 
holding_time_percentage: 4.71% 
n_trades: 21 
win_rate: 66.67% 
cagr: 4.29% 
sharpe: 0.854 
---
ticker: rklb 
total_holding_days: 16 
total_trading_days: 227 
holding_time_percentage: 7.05% 
n_trades: 6 
win_rate: 50.00% 
cagr: -67.05% 
sharpe: -0.0689 
---
ticker: mstr 
total_holding_days: 104 
total_trading_days: 1326 
holding_time_percentage: 7.84% 
n_trades: 62 
win_rate: 74.19% 
cagr: 49.73% 
sharpe: 1.28 
---
ticker: rddt 
total_holding_days: 13 